<a href="https://colab.research.google.com/github/AnupKumarGupta/AbstractiveTextSummarizer/blob/master/Tiatanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn import svm

dataFrameTrain = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Dataset_SVM/train.csv")
dataFrameTest = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Dataset_SVM/test.csv")

In [129]:
if 'PassengerId' in dataFrameTrain.columns: #To resolve errors, if the columns are already dropped
    dataFrameTrain = dataFrameTrain.drop(columns=['PassengerId','Name', 'Ticket', 'Cabin'])
if 'PassengerId' in dataFrameTest.columns:
    dataFrameTest = dataFrameTest.drop(columns=['PassengerId','Name', 'Ticket', 'Cabin'])    

print(dataFrameTrain.isna().any())
# Age and Embarked have null values in training data
print(dataFrameTest.isna().any())
# Age and Fare have null values in training data

Survived    False
Pclass      False
Sex         False
Age          True
SibSp       False
Parch       False
Fare        False
Embarked     True
dtype: bool
Pclass      False
Sex         False
Age          True
SibSp       False
Parch       False
Fare         True
Embarked    False
dtype: bool


In [0]:
X = dataFrameTrain.loc[:, dataFrameTrain.columns != 'Survived'].values
Y = dataFrameTrain.loc[:, dataFrameTrain.columns == 'Survived'].values
X_unseen = dataFrameTest.loc[:, dataFrameTest.columns != 'Survived'].values

In [131]:
#Imputing Age by Mean for training data
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp = imp.fit(X[:,2:3])
X[:,2:3] = imp.transform(X[:,2:3])

#Imputing Age of testing data by mean of training data
meanAgeTrain = np.mean(X[:,2:3])
imp = SimpleImputer(missing_values=np.nan, strategy='constant',fill_value=meanAgeTrain)
imp = imp.fit(X_unseen[:,2:3])
X_unseen[:,2:3] = imp.transform(X_unseen[:,2:3])

#Imputing Age of testing data by Mean of training data
meanFareTrain = np.mean(X[:,5:6])
imp = SimpleImputer(missing_values=np.nan, strategy='constant',fill_value=meanAgeTrain)
imp = imp.fit(X_unseen[:,5:6])
X_unseen[:,5:6] = imp.transform(X_unseen[:,5:6])

#Imputing Embarked by Most Frequent Occurance for training data
imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imp = imp.fit(X[:,6:])
X[:,6:] = imp.transform(X[:,6:])

#checking for NaN values
print(pd.DataFrame(X).isna().any())
print(pd.DataFrame(X_unseen).isna().any())

0    False
1    False
2    False
3    False
4    False
5    False
6    False
dtype: bool
0    False
1    False
2    False
3    False
4    False
5    False
6    False
dtype: bool


In [0]:
# Encoding Sex for training data
labelEncoder = LabelEncoder()
labelEncoder.fit(X[:,1:2].ravel())
X[:,1:2] = labelEncoder.transform(X[:,1:2].ravel()).reshape(X[:,1:2].shape[0],1) 

# Encoding Sex for test data
labelEncoder.fit(X_unseen[:,1:2].ravel())
X_unseen[:,1:2] = labelEncoder.transform(X_unseen[:,1:2].ravel()).reshape(X_unseen[:,1:2].shape[0],1) 

In [0]:
#Encoding Embarkment for training data
labelEncoder = LabelEncoder()
labelEncoder.fit(X[:,6:].ravel())
X[:,6:] = labelEncoder.transform(X[:,6:].ravel()).reshape(X[:,6:].shape[0],1) 

#Encoding Embarkment for test data
labelEncoder.fit(X_unseen[:,6:].ravel())
X_unseen[:,6:] = labelEncoder.transform(X_unseen[:,6:].ravel()).reshape(X_unseen[:,6:].shape[0],1) 

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

In [135]:
clf = svm.SVC(gamma='scale')
clf.fit(X_train, y_train.ravel())

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [136]:
TP=0
FN=0
FP=0
TN=0
for row in range(X_test.shape[0]):
    pred = (clf.predict(X_test[row].reshape(1, -1)))
    if(pred ==1):
      if(y_test[row]==1):
        TP = TP + 1
      else:
        FP = FP +1
    else:
      if(y_test[row]==1):
        FN = FN + 1
      else:
        TN = TN +1
print(TP,FN,FP,TN)
precision = TP/(TP+FP)
recall = TP/(TP+FN)
print(precision, recall)

12 24 3 51
0.8 0.3333333333333333
